# LLM examples

Notebook to illustrate the 

Learning goals: 

- Practice different introductory use cases of LLMs

## Setup

- To run this notebook you will need to create an account in cohere.ai, and get your API Key.
- You also need to install the python package `cohere`.

## Why cohere? 

- It was one of the first providers of LLMs as a service, was already in the market before ChatGPT. 

- Easy to use Python SDK. 

- A free account gives you access to a Trial API key, which allows 100 calls per minute. 

- Similar in structure to other APIs and frameworks (if you learn this, you'll navigate the others easily). 

## Preparing the work

Let's initialize the environment:

In [1]:
import cohere

In [2]:
COHERE_API_KEY = # YOUR API KEY HERE
cohere_client = cohere.Client(COHERE_API_KEY)


## Use case 1: Replacing code

We can use LLMs to take actions for us and avoid the alternative of writing code. 

The example below will ask for a text input by the user. Once introduced, it will classify the language of the text. 

In [3]:
new_text = input("Enter some text")
prompt = "Return the language of the following text: " + new_text

response = cohere_client.chat(
    chat_history=[],
    message=prompt,
)

language = response.text
print(language)

The text is in French.


You might need a more specific prompt, like one that includes "reply only with a single word containing the name of the recognized language" (try that as an exercise). 

The typical workflow would be to use that output as part of the business logic.

In [4]:
if language == "Spanish":
    # Take actions... 
    pass

## Use case 2: Replace an AI model

We can use LLMs to "simulate" the way AI systems work and provide "examples" of what we want to achieve, as part of the prompt. 

This is know as "few-shot", "in-prompt" learning. 

The example below tries to classify the restaurant type based on its name. It does so by preparing a "template" of known inputs and outputs, then asking to complete a last input: 

In [5]:
prompt2="""
Classify the restaurant type based on the restaurant name. 
Complete the following sequence. 
Name: "Leche de Tigre" Type: "Peruvian"
Name: "Le Parisien" Type: "French"
Name: "Tapas & Fiesta" Type: "Spanish"
Name: "Rodizio Grill" Type: "Brazilian"
Name: "Shenzen Garden" Type: "Chinese"
Name: "Bella Napoli" Type: "Italian"
Name: "La taqueria Type:
"""

response = cohere_client.chat(
    chat_history=[],
    message=prompt2,
)

response.text

'Name: "La Taqueria" Type: "Mexican"'

Mind the cost increases with the number of examples. You can get info of the costs:

In [6]:
print(response.json(indent=2))

{
  "text": "Name: \"La Taqueria\" Type: \"Mexican\"",
  "generation_id": "8138b375-e39f-4fb5-9b4e-c267c8b55e14",
  "finish_reason": "COMPLETE",
  "chat_history": [
    {
      "message": "\nClassify the restaurant type based on the restaurant name. \nComplete the following sequence. \nName: \"Leche de Tigre\" Type: \"Peruvian\"\nName: \"Le Parisien\" Type: \"French\"\nName: \"Tapas & Fiesta\" Type: \"Spanish\"\nName: \"Rodizio Grill\" Type: \"Brazilian\"\nName: \"Shenzen Garden\" Type: \"Chinese\"\nName: \"Bella Napoli\" Type: \"Italian\"\nName: \"La taqueria Type:\n",
      "role": "USER"
    },
    {
      "message": "Name: \"La Taqueria\" Type: \"Mexican\"",
      "role": "CHATBOT"
    }
  ],
  "meta": {
    "api_version": {
      "version": "1"
    },
    "billed_units": {
      "input_tokens": 111,
      "output_tokens": 13
    },
    "tokens": {
      "input_tokens": 175,
      "output_tokens": 13
    }
  },
  "response_id": "bbdc5c10-001f-4d99-9948-f80f431ccbdf"
}


## Use case 3: Labeling / Completion

We can use LLMs to add labels to data automatically. 

The next cell loads data from the Barcelona events API. Later we will ask to tag it with a category. 



In [7]:
# Get Barcelona events from API
import requests
import pandas as pd

URL = "https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=877ccf66-9106-4ae2-be51-95a9f6469e4c"
data = requests.get(URL).json()

# Format events as DataFrame
rows = []
for record in data['result']['records']:    
    name = record['name']
    address = record['addresses_road_name']
    number = record['addresses_start_street_number']
    if number:
        address += ', ' + number
    rows.append([name, address])

events = pd.DataFrame(rows, columns=['name', 'address'])

events.sample(5)


,name,address
94,ACTIVITAT ADAPTADA CASAL ESTIU NEXE FUNDACIO -...,"C Escorial, 169"
73,"RUTA KÊNDALS ESPLAI MATINADA PÒRTICS, 15-17 AN...",
22,Divendres lúdics de Khan Jugar,"Plaça del Sortidor, 12"
7,Grup de suport per a la gent gran,"Carrer del Comandant Benítez, 6"
62,Grup d’Ajuda Mutua (GAM) en 1ª Persona,"Carrer de Viladomat, 127"


In [8]:
# Now for each of the events, apply a LLM to infer a category and "fill in" the table.

events = events.sample(5)

prompt = """
We have the name of an event in Barcelona (in Catalan).
Classify the event type based on this information into one of the following categories:
Music, Art, Sport, Cinema, Family, Food, Theater, Exhibitions, Other.
Respond with a single word with the category name. 
Event: 
"""


categories = []
for event in events['name']:
    response = cohere_client.chat(
        chat_history=[],
        message=prompt + event,
    )
    category = response.text
    categories.append(category)

events['category'] = categories
events

,name,address,category
81,"Espectacle ""Monòlegs del Taller de Monodrama""","Carrer de Buenaventura Muñoz, 21",Theater
28,CAMPUS OLÍMPIA POLIESPORTIU 1R - 2N PRIMÀRIA P...,"Numància, 142",Sport
92,CASAL D'ESTIU - BALLEM L'ESTIU! CASAL DE DANSA...,"Carrer de Santa Matilde, 23",Family
61,Taller 'K-Pop',"C Brusi, 51",Music
22,Divendres lúdics de Khan Jugar,"Plaça del Sortidor, 12",Family


## Use case 4: Replacing the user interface

Natural language can be the new interface. 

The example below asks the user to write a text specifying whether to turn the lights on or off.  


In [9]:
from ipywidgets import widgets

checkbox = widgets.Checkbox(
    value=False,
    description='Lights on',
    disabled=False,
    indent=False
)
checkbox

Checkbox(value=False, description='Lights on', indent=False)

In [10]:
prompt3 = """
The user will write a text about turning off or on the lights.
The user might use different words to describe the action or expressions, 
like turn on, turn off, activate, deactivate, switch on, switch off, etc.
The user might also use different words to describe the lights,
like lights, lamps, bulbs, etc.
Return a single word: "true" if the user wants to turn on the lights, and "false" if the user wants to turn them off.
"""

user_input = input("Enter your message")
response = cohere_client.chat(
    message=prompt3 + user_input,
)

print("(Just for debugging, the output text is", response.text, ")")

if response.text == "true":
    checkbox.value = True
if response.text == "false":
    checkbox.value = False

(Just for debugging, the output text is true )


## Use case 5: Structuring data

Use LLMs to extract information and add it to a certain template. 

The next example simulates a text from a health record and how the system would extract the necessary information into a template. 

In [11]:
prompt4 = """
Here is a text simulating an electronic health record.
Extract the information necessary to populate this json object:
{
  "age": X,
  "temperature": X,
  "reason for visit": "X",
  "tests performed": ["X", "X", "X"],
    "diagnosis": "X"
}
If the information is not present, return "Not found".
Respond only with the json object. 
Text:
"""

user_input = input("Enter your message")

response = cohere_client.chat(
    message=prompt4 + user_input,
)

print(response.text)

{
  "age": 23,
  "temperature": "Not found",
  "reason for visit": "Stomach ache",
  "tests performed": ["Radiography"],
  "diagnosis": "Possible virus"
}


## Use case: Chatbots

This iexample illustrates the use of a chatbot, where the intent to be fulfilled is accomplished after several messages, and each message needs to include the previous chat history. 

Let's do it in the wrong way first: 

In [12]:
response1 = cohere_client.chat(message="Who discovered gravity?")
response2 = cohere_client.chat(message="What year was he born?")

print(response2.text)

I can answer that question if you tell me the person you are referring to.


In [13]:
response1.chat_history

[Message_User(message='Who discovered gravity?', tool_calls=None, role='USER'),
 Message_Chatbot(message='The English scientist Sir Isaac Newton discovered gravity in the late 17th century. He introduced the concept in his work "Philosophiæ Naturalis Principia Mathematica," published on July 5, 1687. In this groundbreaking book, Newton presented his three laws of motion, which include the law of universal gravitation.\n\nNewton\'s law of universal gravitation states that every particle of matter in the universe attracts every other particle with a force that is directly proportional to the product of their masses and inversely proportional to the square of the distance between them. Mathematically, this can be represented as:\n\nF = G * (m1 * m2) / r^2\n\nwhere F is the force between two objects, m1 and m2 are their masses, r is the distance between their centers, and G is the gravitational constant.\n\nNewton\'s discovery of gravity was a significant breakthrough in our understanding 

We need to add the previous chat history, that was stored already in the response json:

In [14]:
response = cohere_client.chat(
    chat_history=response1.chat_history,
    message="What year was he born?",
)
print(response.text)


Sir Isaac Newton was born in Woolsthorpe, Lincolnshire, England, on December 25, 1642, according to the Julian calendar in use at the time, which is equivalent to January 4, 1643, in the Gregorian calendar that we use today. He was born during a period of great political and scientific change, and his contributions to the fields of mathematics, physics, and astronomy would go on to have a profound impact on the scientific revolution.

Newton came from a humble background, and he showed great intellectual promise from an early age. Despite facing challenges and setbacks, including the closure of his school due to the plague, he went on to study at the University of Cambridge, where he made many of his groundbreaking discoveries.

His birth year, 1642, is often considered a pivotal year in the history of science, as it also marks the year of Galileo Galilei's death and the year in which René Descartes published his influential work "Meditations on First Philosophy." This coincidence of e